[toc]

# Pytorch KL散度和交叉熵

在关于知识蒸馏的代码中，常常会见到使用 KLDiv 函数。

In [4]:
import torch 
import torch.nn.functional as F

def loss_fn_kd(outputs, labels, teacher_outputs, params):
    """
    Compute the knowledge-distillation (KD) loss given outputs, labels.
    "Hyperparameters": temperature and alpha
    NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
    and student expects the input tensor to be log probabilities! See Issue #2
    """
    alpha = params.alpha
    T = params.temperature
    KD_loss = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1),
                             F.softmax(teacher_outputs/T, dim=1)) * (alpha * T * T) +\
                             F.cross_entropy(outputs, labels) * (1. - alpha)
 
    return KD_loss

这其中有两个奇怪的地方：
1. 为什么要使用 nn.KLDivLoss，使用 nn.CrossEntropy 不可以吗？
2. 为什么要对 nn.KLDivLoss 的第一个参数取 log，而不对第二个参数取 log

## 问题一

从道理上讲，最小化 crossentropy 和 最小化 kl divergence 是等价的，因为有下列等式成立：

Entropy(p) + KL Divergence(p||q) = CrossEntropy(p||q)

其中，

Entropy(p) = - p(x) log p(x)

KL Divergence(p||q) = - p(x) log q(x) / p(x) = p(x) log p(x) - p(x) log q(x)

CrossEntropy(p || q) = - p(x) log q(x)

由于 Entropy 对于一个固定的 p(x) 来说是固定，因此减小 KL Divergence 和 减小 CrossEntropy 是等价的。

那为啥在网上的实现中都是最小化 KL Divergence，而不是最小化 CrossEntropy 呢？这个实际上是软件实现上的原因。

因为 Pytorch 中的 nn.CrossEntropy 只支持 p 是 hard label，而 q 是 soft label 的情况。即 nn.CrossEntropy 不支持两个都是 soft label 的情况。

而 nn.KLDivLoss 支持两个都是 soft_label 的情况。因此大家倾向于使用 nn.KLDivLoss。当然，理论上说，如果可以手动实现一个支持 p 和 q 都是 soft label 的 CrossEntropy 函数也是可以的。

## 问题二

这个问题和 `nn.KLDivLoss` 的功能有关。从文档中可以看到，`nn.KLDivLoss` 计算的是

$$
1(\mathrm{x}, \mathrm{y})=\mathrm{L}=\left\{l_{1}, \ldots, \mathrm{l}_{\mathrm{N}}\right\}, \quad l_{\mathrm{n}}=\mathrm{y}_{\mathrm{n}} \cdot\left(\log \mathrm{y}_{\mathrm{n}}-\mathrm{x}_{\mathrm{n}}\right)
$$

这里的 $y$ 对应上述公式中的 $p(x)$，而 $x$ 对应上述公式中的 $q(x)$。可以看到，`nn.KLDivLoss` 计算时，并不会对 x 取 log。因此为了正确计算 KL Divergence，需要手动添加 log。这也就是为什么第一个参数会添加 log 的原因。

# References

1. [(4条消息)知识蒸馏（Knowledge Distillation）_AI Flash-CSDN博客](https://blog.csdn.net/nature553863/article/details/80568658)
2. [KLDivLoss — PyTorch 1.6.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html?highlight=nn%20kldiv#torch.nn.KLDivLoss)